## Setup

In [1]:
!git clone https://github.com/GNNatan/SemNov_AML_DAAI_23-24.git

Cloning into 'SemNov_AML_DAAI_23-24'...
remote: Enumerating objects: 1121, done.
remote: Counting objects: 100% (586/586), done.
remote: Compressing objects: 100% (359/359), done.
remote: Total 1121 (delta 286), reused 432 (delta 216), pack-reused 535
Receiving objects: 100% (1121/1121), 3.26 GiB | 24.37 MiB/s, done.
Resolving deltas: 100% (358/358), done.
Updating files: 100% (742/742), done.


In [1]:
%cd SemNov_AML_DAAI_23-24/

/content/SemNov_AML_DAAI_23-24


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.5/881.5 kB 31.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-

In [3]:
!pip install -r saliency/requirements.txt

  Preparing metadata (setup.py) ... done
  Created wheel for cprint: filename=cprint-1.2.2-py3-none-any.whl size=2519 sha256=8a3b53297d7234d2b035bccacc7d3697860e2179ed55cfd30837214c4ca6fef3
  Stored in directory: /root/.cache/pip/wheels/e8/c7/77/b74864ee8de3abb693e3be291549a29c6ff242ce66801b1622
Successfully built cprint


In [4]:
!sh download_data.sh

============Downloading ModelNet40 + OOD Splits in 
--2024-06-14 11:23:18--  https://www.dropbox.com/s/c2x3h59nxprjs21/modelnet40_normal_resampled.tar?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/c2x3h59nxprjs21/modelnet40_normal_resampled.tar [following]
--2024-06-14 11:23:18--  https://www.dropbox.com/s/dl/c2x3h59nxprjs21/modelnet40_normal_resampled.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc22bbc5a95f8d67aa6a75720912.dl.dropboxusercontent.com/cd/0/get/CUzNnzSkeyrD7u1xYPdwbP71p1-hZDiYvoBbX3jcBgVALjDHBV8onwoOhXVhCggtDlYP-P52bks8RUa4vIg06peWXjSMp0BVBS0-8kzfCX5W2xZoidKp7013-5NatHwwLhvNyX_4vSxlyro5nSxmdYNZ/file?dl=1# [following]
--2024-06-14 11:23:19--  https://uc22bbc5a95f8d67aa6a75720912.dl.dropboxusercontent.com/

In [2]:
import torch
import os
import h5py
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from random import sample
from tqdm import tqdm
from mpl_toolkits.mplot3d import proj3d
from sklearn.metrics import roc_curve
from notebooks.utils import lab_to_text_sr1, lab_to_text_sr2, load_from_file, render_cloud
import saliency.render_utils as saliency

In [3]:
SR1 = {
    4: 0,  # chair
    8: 1,  # shelf
    7: 2,  # door
    12: 3,  # sink
    13: 4  # sofa
}

SR2 = {
    10: 0,  # bed
    14: 1,  # toilet
    5: 2,  # desk
    6: 3,  # display
    9: 2  # table
}

SR3 = {
    0: 404,  # bag
    1: 404,  # bin
    2: 404,  # box
    3: 404,  # cabinet
    11: 404  # pillow
}

test_file = h5py.File('3D_OS_release_data/ScanObjectNN/h5_files/main_split/test_objectdataset.h5', 'r')
train_file = h5py.File('3D_OS_release_data/ScanObjectNN/h5_files/main_split/training_objectdataset.h5', 'r')

test_labels  = test_file['label'][:]
train_labels = train_file['label'][:]

test_clouds = test_file['data'][:]
train_clouds = train_file['data'][:]

all_labels = np.hstack((train_labels, test_labels))
all_clouds = np.vstack((train_clouds, test_clouds))

SR1_indices = [index for index, value in enumerate(all_labels) if value in SR1.keys()]
SR2_indices = [index for index, value in enumerate(all_labels) if value in SR2.keys()]
SR3_indices = [index for index, value in enumerate(all_labels) if value in SR3.keys()]

SR1_clouds = all_clouds[SR1_indices]
SR2_clouds = all_clouds[SR2_indices]
SR3_clouds = all_clouds[SR3_indices]

SR1_labels = [SR1[label] for label in all_labels[SR1_indices]]
SR2_labels = [SR2[label] for label in all_labels[SR2_indices]]
SR3_labels = [404 for _ in SR3_indices]

all_indices_sr1 = np.hstack((SR1_indices, SR2_indices, SR3_indices))
all_labels_sr1 = all_labels[all_indices_sr1]

all_indices_sr2 = np.hstack((SR2_indices, SR1_indices, SR3_indices))
all_labels_sr2 = all_labels[all_indices_sr2]

In [4]:
def acc_check(src, truth, preds):
  acc = 0
  for i in range(len(src)):
    if truth[i] == preds[i]:
      acc += 1

  print(acc/len(SRC))

In [5]:
def find_threshold(src_labels, tar1_labels, tar2_labels, scores):
  ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

  fpr, tpr, thr = roc_curve(ood_truth, scores, pos_label=1)

  for i in range(len(tpr)-1, 0, -1):
    if tpr[i] < 0.95:
      break

  print("FPR at 95:", fpr[i])
  return thr[i+1]

In [6]:
def classify_ood(truth, scores, preds, threshold):
  correct = list()
  wrong = list()
  preds_ood = list()
  fp = 0

  for i in range(len(truth)):
    pred = preds[i]
    if scores[i] < threshold:
      pred = torch.tensor(404)
    if pred == truth[i]:
      correct.append(i)
    else:
      wrong.append(i)
      if truth[i] == torch.tensor(404): # false positive
        fp += 1
    preds_ood.append(pred)

  print("Correctly classified", len(correct), "samples.",
        "\nMisclassified", len(wrong), "samples.",
        "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check
  return correct, wrong, preds_ood

In [7]:
def write_output(data_name, folder_name, truth, preds):
  OUTPUT_FOLDER = 'outputs/classification_results'
  if not os.path.exists(f'{OUTPUT_FOLDER}/{data_name}/{folder_name}'):
    os.makedirs(f'{OUTPUT_FOLDER}/{data_name}/{folder_name}')

  summary = {}


  for sample in tqdm(range(len(truth))):
    true_class = lab_to_text[truth[sample].item()]
    pred_class = lab_to_text[preds[sample].item()]
    title = f"{true_class}-{pred_class}"
    file = open(f'{OUTPUT_FOLDER}/{data_name}/{folder_name}/{title}.txt', 'a+')
    file.write(str(sample)+'\n')
    file.close()
    if true_class == pred_class:
      if f"{true_class}-correct" not in summary.keys():
        summary[f"{true_class}-correct"] = 1
      else:
        summary[f"{true_class}-correct"] += 1
    else:
      if f"{true_class}-{pred_class}" not in summary.keys():
        summary[f"{true_class}-{pred_class}"] = 1
      else:
        summary[f"{true_class}-{pred_class}"] += 1

  file = open(f'{OUTPUT_FOLDER}/{data_name}/{folder_name}/summary.txt', 'a+')
  file.write(str(dict(sorted(summary.items(), key=lambda item: item[1]))))
  file.close()

## PointNet++

### SR1

In [9]:
DATA_NAME = 'PN2_CE_SR1'
SRC = SR1_labels
TAR1 = SR2_labels
TAR2 = SR3_labels
lab_to_text = lab_to_text_sr1

#### Discriminative method

In [10]:
##### DISCRIMINATIVE - setup
src_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/src_logits.pt')
tar1_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_logits.pt')
tar2_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_logits.pt')

src_MSP_scores, src_MSP_pred = F.softmax(src_logits, dim=1).max(1)
tar1_MSP_scores, tar1_MSP_pred = F.softmax(tar1_logits, dim=1).max(1)
tar2_MSP_scores, tar2_MSP_pred = F.softmax(tar2_logits, dim=1).max(1)

In [11]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [12]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [13]:
acc_check(SRC, truth, preds_MSP)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_MSP)
correct, wrong, preds_ood = classify_ood(truth, scores_MSP, preds_MSP, threshold)
write_output(DATA_NAME, 'MSP', truth, preds_ood)

0.7880478087649403
FPR at 95: 0.8048929663608563
Correctly classified 1287 samples. 
Misclassified 1603 samples. 
FPR: 0.8048929663608563


100%|██████████| 2890/2890 [00:00<00:00, 15571.18it/s]


#### Distance based method

In [14]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")
src_feats = load_from_file(f'outputs/feats/{DATA_NAME}.pt')

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [15]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_ids])


In [16]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [17]:
acc_check(SRC, truth, preds_L2)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_L2)
correct, wrong, preds_ood = classify_ood(truth, scores_L2, preds_L2, threshold)
write_output(DATA_NAME, 'L2', truth, preds_ood)

0.7800796812749003
FPR at 95: 0.8379204892966361
Correctly classified 1223 samples. 
Misclassified 1667 samples. 
FPR: 0.8379204892966361


100%|██████████| 2890/2890 [00:00<00:00, 15849.71it/s]


### SR2

In [18]:
DATA_NAME = 'PN2_CE_SR2'
SRC = SR2_labels
TAR1 = SR1_labels
TAR2 = SR3_labels
lab_to_text = lab_to_text_sr2

#### Discriminative method

In [19]:
##### DISCRIMINATIVE - setup
src_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/src_logits.pt')
tar1_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_logits.pt')
tar2_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_logits.pt')

src_MSP_scores, src_MSP_pred = F.softmax(src_logits, dim=1).max(1)
tar1_MSP_scores, tar1_MSP_pred = F.softmax(tar1_logits, dim=1).max(1)
tar2_MSP_scores, tar2_MSP_pred = F.softmax(tar2_logits, dim=1).max(1)

In [20]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [21]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [22]:
acc_check(SRC, truth, preds_MSP)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_MSP)
correct, wrong, preds_ood = classify_ood(truth, scores_MSP, preds_MSP, threshold)
write_output(DATA_NAME, 'MSP', truth, preds_ood)

0.8375634517766497
FPR at 95: 0.8691722169362512
Correctly classified 911 samples. 
Misclassified 1979 samples. 
FPR: 0.8691722169362512


100%|██████████| 2890/2890 [00:00<00:00, 22638.76it/s]


#### Distance based method

In [23]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")
src_feats = load_from_file(f'outputs/feats/{DATA_NAME}.pt')

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [24]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_L2_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_L2_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_L2_ids])

In [25]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [26]:
acc_check(SRC, truth, preds_L2)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_L2)
correct, wrong, preds_ood = classify_ood(truth, scores_L2, preds_L2, threshold)
write_output(DATA_NAME, 'L2', truth, preds_ood)

0.8362944162436549
FPR at 95: 0.807802093244529
Correctly classified 1041 samples. 
Misclassified 1849 samples. 
FPR: 0.807802093244529


100%|██████████| 2890/2890 [00:00<00:00, 18341.98it/s]


## DGCNN

### SR1

In [27]:
DATA_NAME = 'DGCNN_CE_SR1'
SRC = SR1_labels
TAR1 = SR2_labels
TAR2 = SR3_labels
lab_to_text = lab_to_text_sr1

#### Discriminative method

In [28]:
##### DISCRIMINATIVE - setup
src_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/src_logits.pt')
tar1_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_logits.pt')
tar2_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_logits.pt')

src_MSP_scores, src_MSP_pred = F.softmax(src_logits, dim=1).max(1)
tar1_MSP_scores, tar1_MSP_pred = F.softmax(tar1_logits, dim=1).max(1)
tar2_MSP_scores, tar2_MSP_pred = F.softmax(tar2_logits, dim=1).max(1)

In [29]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [30]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [31]:
acc_check(SRC, truth, preds_MSP)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_MSP)
correct, wrong, preds_ood = classify_ood(truth, scores_MSP, preds_MSP, threshold)
write_output(DATA_NAME, 'MSP', truth, preds_ood)

0.7243027888446215
FPR at 95: 0.9015290519877676
Correctly classified 1047 samples. 
Misclassified 1843 samples. 
FPR: 0.9015290519877676


100%|██████████| 2890/2890 [00:00<00:00, 22901.85it/s]


#### Distance based method

In [32]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")
src_feats = load_from_file(f'outputs/feats/{DATA_NAME}.pt')

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [33]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_ids])


In [34]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [35]:
acc_check(SRC, truth, preds_L2)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_L2)
correct, wrong, preds_ood = classify_ood(truth, scores_L2, preds_L2, threshold)
write_output(DATA_NAME, 'L2', truth, preds_ood)

0.7298804780876494
FPR at 95: 0.8538226299694189
Correctly classified 1125 samples. 
Misclassified 1765 samples. 
FPR: 0.8538226299694189


100%|██████████| 2890/2890 [00:00<00:00, 19240.26it/s]


### SR2

In [36]:
DATA_NAME = 'DGCNN_CE_SR2'
SRC = SR2_labels
TAR1 = SR1_labels
TAR2 = SR3_labels
lab_to_text = lab_to_text_sr2

#### Discriminative method

In [37]:
##### DISCRIMINATIVE - setup
src_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/src_logits.pt')
tar1_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_logits.pt')
tar2_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_logits.pt')

src_MSP_scores, src_MSP_pred = F.softmax(src_logits, dim=1).max(1)
tar1_MSP_scores, tar1_MSP_pred = F.softmax(tar1_logits, dim=1).max(1)
tar2_MSP_scores, tar2_MSP_pred = F.softmax(tar2_logits, dim=1).max(1)

In [38]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [39]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [40]:
acc_check(SRC, truth, preds_MSP)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_MSP)
correct, wrong, preds_ood = classify_ood(truth, scores_MSP, preds_MSP, threshold)
write_output(DATA_NAME, 'MSP', truth, preds_ood)

0.6789340101522843
FPR at 95: 0.9005708848715509
Correctly classified 731 samples. 
Misclassified 2159 samples. 
FPR: 0.9005708848715509


100%|██████████| 2890/2890 [00:00<00:00, 13366.14it/s]


#### Distance based method

In [41]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")
src_feats = load_from_file(f'outputs/feats/{DATA_NAME}.pt')

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [42]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_ids])


In [43]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
preds_L2 = torch.where(preds_L2==4, torch.tensor(2), preds_L2)
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [44]:
acc_check(SRC, truth, preds_L2)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_L2)
correct, wrong, preds_ood = classify_ood(truth, scores_L2, preds_L2, threshold)
write_output(DATA_NAME, 'L2', truth, preds_ood)

0.6979695431472082
FPR at 95: 0.8920076117982874
Correctly classified 754 samples. 
Misclassified 2136 samples. 
FPR: 0.8920076117982874


100%|██████████| 2890/2890 [00:00<00:00, 25347.68it/s]


## OpenShape

### SR1

In [45]:
DATA_NAME = 'OpenShape_SR1'
SRC = SR1_labels
TAR1 = SR2_labels
TAR2 = SR3_labels
SRC_indices = SR1_indices
TAR1_indices = SR2_indices
TAR2_indices = SR3_indices
lab_to_text = lab_to_text_sr1

#### Discriminative method

In [46]:
##### DISCRIMINATIVE - setup
img_scores = load_from_file(f'outputs/tensors/{DATA_NAME}/img_scores.pt')
txt_scores = load_from_file(f'outputs/tensors/{DATA_NAME}/text_scores.pt')

openshape_scores = img_scores + txt_scores

src_logits = torch.tensor(np.array([openshape_scores[i] for i in SRC_indices])).squeeze(1)
tar1_logits = torch.tensor(np.array([openshape_scores[i] for i in TAR1_indices])).squeeze(1)
tar2_logits = torch.tensor(np.array([openshape_scores[i] for i in TAR2_indices])).squeeze(1)

src_MSP_scores, src_MSP_pred = src_logits.max(1)
tar1_MSP_scores, tar1_MSP_pred = tar1_logits.max(1)
tar2_MSP_scores, tar2_MSP_pred = tar2_logits.max(1)

In [47]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [48]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [49]:
acc_check(SRC, truth, preds_MSP)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_MSP)
correct, wrong, preds_ood = classify_ood(truth, scores_MSP, preds_MSP, threshold)
write_output(DATA_NAME, 'MSP', truth, preds_ood)

0.8589641434262948
FPR at 95: 0.6709480122324158
Correctly classified 1583 samples. 
Misclassified 1307 samples. 
FPR: 0.6709480122324158


100%|██████████| 2890/2890 [00:00<00:00, 23490.81it/s]


#### Distance based method

In [50]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [51]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_ids])


In [52]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [53]:
acc_check(SRC, truth, preds_L2)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_L2)
correct, wrong, preds_ood = classify_ood(truth, scores_L2, preds_L2, threshold)
write_output(DATA_NAME, 'L2', truth, preds_ood)

0.9139442231075697
FPR at 95: 0.8159021406727829
Correctly classified 1401 samples. 
Misclassified 1489 samples. 
FPR: 0.8159021406727829


100%|██████████| 2890/2890 [00:00<00:00, 24217.21it/s]


### SR2

In [54]:
DATA_NAME = 'OpenShape_SR2'
SRC = SR2_labels
TAR1 = SR1_labels
TAR2 = SR3_labels
SRC_indices = SR2_indices
TAR1_indices = SR1_indices
TAR2_indices = SR3_indices
lab_to_text = lab_to_text_sr2

#### Discriminative method

In [55]:
##### DISCRIMINATIVE - setup
img_scores = load_from_file(f'outputs/tensors/{DATA_NAME}/img_scores.pt')
txt_scores = load_from_file(f'outputs/tensors/{DATA_NAME}/text_scores.pt')

openshape_scores = img_scores + txt_scores

src_logits = torch.tensor(np.array([openshape_scores[i] for i in SRC_indices])).squeeze(1)
tar1_logits = torch.tensor(np.array([openshape_scores[i] for i in TAR1_indices])).squeeze(1)
tar2_logits = torch.tensor(np.array([openshape_scores[i] for i in TAR2_indices])).squeeze(1)

# merge desk and table prediction into same class for SR2
src_table = src_logits[:, 2] + src_logits[:, 4]
tar1_table = tar1_logits[:, 2] + tar1_logits[:, 4]
tar2_table = tar2_logits[:, 2] + tar2_logits[:, 4]

src_logits = torch.cat((src_logits[:, :2], src_table.unsqueeze(1), src_logits[:, 3].unsqueeze(1)), dim=1)
tar1_logits = torch.cat((tar1_logits[:, :2], tar1_table.unsqueeze(1), tar1_logits[:, 3].unsqueeze(1)), dim=1)
tar2_logits = torch.cat((tar2_logits[:, :2], tar2_table.unsqueeze(1), tar2_logits[:, 3].unsqueeze(1)), dim=1)

src_MSP_scores, src_MSP_pred = src_logits.max(1)
tar1_MSP_scores, tar1_MSP_pred = tar1_logits.max(1)
tar2_MSP_scores, tar2_MSP_pred = tar2_logits.max(1)

In [56]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [57]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [58]:
acc_check(SRC, truth, preds_MSP)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_MSP)
correct, wrong, preds_ood = classify_ood(truth, scores_MSP, preds_MSP, threshold)
write_output(DATA_NAME, 'MSP', truth, preds_ood)

0.5203045685279187
FPR at 95: 0.9533777354900095
Correctly classified 501 samples. 
Misclassified 2389 samples. 
FPR: 0.9533777354900095


100%|██████████| 2890/2890 [00:00<00:00, 17091.00it/s]


#### Distance based method

In [59]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [60]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_ids])


In [61]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [62]:
acc_check(SRC, truth, preds_L2)
threshold = find_threshold(src_labels, tar1_labels, tar2_labels, scores_L2)
correct, wrong, preds_ood = classify_ood(truth, scores_L2, preds_L2, threshold)
write_output(DATA_NAME, 'L2', truth, preds_ood)

0.5647208121827412
FPR at 95: 0.8810656517602283
Correctly classified 675 samples. 
Misclassified 2215 samples. 
FPR: 0.8810656517602283


100%|██████████| 2890/2890 [00:00<00:00, 25714.79it/s]


## Rendering

In [64]:
lab_to_text_all = {
    4: 'chair',
    8: 'shelf',
    7: 'door',
    12: 'sink',
    13: 'sofa',
    10: 'bed',
    14: 'toilet',
    5: 'desk',
    6: 'display',
    9: 'table',
    0: 'bag',
    1: 'bin',
    2: 'box',
    3: 'cabinet',
    11: 'pillow'
}

In [65]:
device, args, model = saliency.load_model()

In [106]:
args.drop_neg = True
args.num_steps = 10

In [108]:
SAL_FOLDER = 'outputs/renders/sal'
if not os.path.exists(SAL_FOLDER):
  os.makedirs(SAL_FOLDER)
index = np.random.choice(np.fromfile(f'outputs/classification_results/DGCNN_CE_SR2/MSP/table-bed.txt', dtype=int, sep='\n'))
cloud_index = all_indices_sr2[index]
label = all_labels[cloud_index]
title = lab_to_text_all[label]
cloud = all_clouds[cloud_index]
adv, colors = saliency.calc_saliency(cloud, label, args, model, device)
colors = saliency.normalize_quantile(colors)
render_cloud(cloud, filename=f"{SAL_FOLDER}/{cloud_index}.png", title = title)
render_cloud(cloud, colors=colors, filename=f"{SAL_FOLDER}/{cloud_index}-sal.png", title = title)
render_cloud(adv.detach().numpy(), filename=f"{SAL_FOLDER}/{cloud_index}-adv.png", title = title)